In [6]:
import numpy as np
from astropy.timeseries import LombScargle
import pandas as pd
import os
import seaborn
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import clear_output
from tqdm.notebook import *
import IProgress

seaborn.set()
cwd = os.getcwd()

In [7]:
star_data = pd.read_table(f'{cwd}/star_data.txt', delim_whitespace=True) # File with RA, Dec, Period, dPeriod

# star_data.to_csv(f'{cwd}/star_data.txt', sep=' ', index=False)

PHOTOMETRY_FILE_CONTAINER = [f'{cwd}/Photometry/I', f'{cwd}/Photometry/V'] # t, mag, dmag

cols = ['t', 'y', 'dy']

In [8]:
star_data = star_data[star_data['V'] > 0]
star_data = star_data.reset_index(drop=True)

In [11]:
smc = pd.read_table(f'{cwd}/smcred.dat', delim_whitespace=True)
smc['Subtype'] = smc['Subtype'].astype('str')
smc = smc[smc['Subtype'] == 'BLHer']

smc['V'] = smc['V'] - smc['A_V']
smc['I'] = smc['I'] - smc['A_I']


lmc = pd.read_table(f'{cwd}/lmcred.dat', delim_whitespace=True)
lmc['Subtype'] = lmc['Subtype'].astype('str')
lmc = lmc[lmc['Subtype'] == 'BLHer']

lmc['V'] = lmc['V'] - lmc['A_V']
lmc['I'] = lmc['I'] - lmc['A_I']

g = pd.concat([smc, lmc])

star_data['ID'] = star_data['ID'].astype('str')
g['ID'] = g['ID'].astype('str')

for i in range(len(star_data)):
    a = g.loc[g['ID'] == star_data['ID'].iloc[i]]
    if len(a['V']) == 0:
        star_data = star_data.drop(i)
    elif len(a['V']) == 1:
#         print(a['V'])
        star_data['I'].iloc[i] = a['I']
        star_data['V'].iloc[i] = a['V']
star_data = star_data.reset_index(drop=True)

C:\Users\selim\AppData\Local\Temp/ipykernel_2784/2763795636.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_data['I'].iloc[i] = a['I']
C:\Users\selim\AppData\Local\Temp/ipykernel_2784/2763795636.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_data['V'].iloc[i] = a['V']


In [12]:
star_data

,ID,I,V,P_1,dP_1
0,OGLE-LMC-T2CEP-006,17.8800,18.2840,1.087928,0.000002
1,OGLE-LMC-T2CEP-007,17.7650,18.1770,1.242690,0.000002
2,OGLE-LMC-T2CEP-008,17.5795,18.2015,1.746120,0.000003
3,OGLE-LMC-T2CEP-009,17.6150,18.1340,1.761347,0.000003
4,OGLE-LMC-T2CEP-010,17.7205,18.2535,1.502952,0.000002
...,...,...,...,...,...
104,OGLE-SMC-T2CEP-42,18.1190,18.7460,1.487513,0.000003
105,OGLE-SMC-T2CEP-45,18.0315,18.5065,1.425774,0.000003
106,OGLE-SMC-T2CEP-51,18.6705,19.2805,1.065770,0.000004
107,OGLE-SMC-T2CEP-52,18.1170,18.7340,1.746251,0.000003


In [24]:
def cos_series(phase, cosines, phis, freq, const): # Evaluates a cosine series
    PI = np.pi
    mag = []
    n_terms=len(cosines)
    for _t in phase:
        
        a_terms = np.dot(cosines, [np.cos(2*PI*freq*(i+1)*_t + phis[i]) for i in range(n_terms)])
        M = np.sum(a_terms) + const
        mag.append(M)
        
    return np.array(mag)

def sin_series(phase, sines, phis, freq, const): # Evaluates a sine series
    PI = np.pi
    mag = []
    n_terms=len(sines)
    for _t in phase:
        
        a_terms = np.dot(sines, [np.sin(2*PI*freq*(i+1)*_t + phis[i]) for i in range(n_terms)])
        M = np.sum(a_terms) + const
        mag.append(M)
        
    return np.array(mag)

def skewness(a_k, phi_k, period):
    
    phase_domain = np.linspace(0, period, 10000)
    freq = 1/period
    
    y = cos_series(phase_fit, a_k, phi_k, freq, 0)
    y_prime = -2*np.pi*freq*sin_series(phase_fit, a_k, phi_k, freq, 0)
    
    rising_branch = np.argwhere(y_prime > 0)
    descending_branch = np.argwhere(y_prime < 0)
    skewness = rising_branch.size/descending_branch.size
    
#     print(f'Skewness: {round(skewness, 4)}')
    return round(skewness, 4)
    
def acuteness(a_k, phi_k, period):
    
    phase_ = np.linspace(0, period, 10000)
    freq = 1/period
    
    y = cos_series(phase_, a_k, phi_k, freq, 0)
    half_maximum = max(y)/2
        
    greater = np.argwhere(y > half_maximum)
    lesser = np.argwhere(y < half_maximum)
    acuteness = greater.size/lesser.size
    
#     print(f'Acuteness: {round(acuteness, 4)}')
    return round(acuteness, 4)

def amplitude(a_k, phi_k, period):
    
    phase_ = np.linspace(0, period, 10000)
    freq = 1/period
    
    y = cos_series(phase_, a_k, phi_k, freq, 0)
    amp = max(y) - min(y)
    
    return amp

Fitting Fourier cosine series

In [25]:
import time

P = star_data['P_1']
dP = star_data['dP_1']

futures = [ [] for p in P ]
names = []
for i in tqdm(range(len(star_data))):
#     clear_output(wait=True)
#     print(star_data["ID"].iloc[i])
    period = star_data['P_1'].iloc[i]
    freq = 1/period
    
    def func(x, a_0, a_1, phi_1, a_2, phi_2, a_3, phi_3): # Change fourier fit depth by just adding more coefficents
        pi = np.pi
        a_k = [a_1, a_2, a_3]
        phi_k = [phi_1, phi_2, phi_3]
        trig_part = [np.cos(2*(i+1)*pi*freq*x + phi_k[i]) for i in range(len(a_k))]
        return a_0 + np.dot(a_k, trig_part)
    try:
        file = f'{star_data["ID"].iloc[i]}.dat'
        path = os.path.join(PHOTOMETRY_FILE_CONTAINER[0], file)
        df = pd.read_table(path, delim_whitespace=True, names = cols)

        t = df['t']
        y = df['y']
        dy = df['dy']

        phase = t % period
        phase_fit = np.linspace(0, period, 10000)

        coef, pcov = curve_fit(func, phase, y, sigma=dy, method='dogbox')

        amplitudes = coef[1::2]
        shifts = coef[2::2]
        const = coef[0]

        perr = np.sqrt(np.diag(pcov))
        s_k = skewness(amplitudes, shifts, period)
        a_c = acuteness(amplitudes, shifts, period)

#         plt.errorbar(phase, y - const, dy, fmt='.', color='red')
#         plt.plot(phase_fit, cos_series(phase_fit, amplitudes, shifts, freq, 0), color='black')
#         plt.xlabel('Phase (days)')
#         plt.ylabel('Magnitude')
#         plt.show()
        amp = amplitude(amplitudes, shifts, period)

        for c in coef:
            futures[i].append(c)
        for e in perr:
            futures[i].append(e)
            
        futures[i].append(s_k)
        futures[i].append(a_c)
        futures[i].append(amp)
        futures[i].append(period)
        futures[i].append(np.log10(period))
                
        names.append(star_data["ID"].iloc[i])
        
        file = f'{star_data["ID"].iloc[i]}.dat'
        path = os.path.join(PHOTOMETRY_FILE_CONTAINER[1], file)
        df = pd.read_table(path, delim_whitespace=True, names = cols)

        t = df['t']
        y = df['y']
        dy = df['dy']

        phase = t % period
        phase_fit = np.linspace(0, period, 10000)

        coef, pcov = curve_fit(func, phase, y, sigma=dy, method='dogbox')

        amplitudes = coef[1::2]
        shifts = coef[2::2]
        const = coef[0]

        perr = np.sqrt(np.diag(pcov))
        s_k = skewness(amplitudes, shifts, period)
        a_c = acuteness(amplitudes, shifts, period)
        amp = amplitude(amplitudes, shifts, period)
        
        
        for c in coef:
            futures[i].append(c)
        for e in perr:
            futures[i].append(e)
            
        futures[i].append(s_k)
        futures[i].append(a_c)
        futures[i].append(amp)
        futures[i].append(star_data['V'].iloc[i])
        futures[i].append(star_data['I'].iloc[i])
        
    except FileNotFoundError:
        star_data = star_data.drop(i)

  0%|          | 0/108 [00:00<?, ?it/s]

C:\Users\selim\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\selim\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\selim\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\selim\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [37]:
Columns = ['i_a0', 'i_a1', 'i_a2', 'i_a3', 'i_phi1', 'i_phi2', 'i_phi3',
   'di_a0', 'di_a1', 'di_a2', 'di_a3', 'di_phi1', 'di_phi2', 'di_phi3',
  'i_skewness', 'i_acuteness', 'i_amp', 'period', 'log(P)', 
           'v_va0', 'v_a1', 'v_a2', 'v_a3', 'v_phi1', 'v_phi2', 'v_phi3',
   'dv_a0', 'dv_a1', 'dv_a2', 'dv_a3', 'dv_phi1', 'dv_phi2', 'dv_phi3',
  'v_skewness', 'v_acuteness', 'v_amp', 'V', 'I']

In [40]:
dat = pd.DataFrame(data=futures, columns=Columns)
dat = dat.dropna()
dat.index = names
dat.to_csv('Observed_parameters.dat', sep=' ')

In [41]:
dat

,i_a0,i_a1,i_a2,i_a3,i_phi1,i_phi2,i_phi3,di_a0,di_a1,di_a2,...,dv_a2,dv_a3,dv_phi1,dv_phi2,dv_phi3,v_skewness,v_acuteness,v_amp,V,I
OGLE-LMC-T2CEP-006,18.024930,-0.225734,-0.734991,-0.052706,0.037669,-0.019924,0.577291,0.001092,0.001552,0.006819,...,0.015192,0.004961,0.062776,0.005366,0.160365,1.3480,0.5635,0.801192,18.2840,17.8800
OGLE-LMC-T2CEP-007,18.029705,0.224761,0.239149,-0.055068,1.807476,0.016619,5.246076,0.001059,0.001523,0.006556,...,0.008407,0.003162,0.037096,0.003195,0.114050,1.3491,0.4110,0.794525,18.1770,17.7650
OGLE-LMC-T2CEP-008,17.808965,-0.212514,-1.056608,-0.045962,0.094159,-0.032290,-0.402964,0.002111,0.003040,0.013751,...,0.029355,0.009890,0.167365,0.009957,0.185697,1.2031,0.4957,0.708760,18.2015,17.5795
OGLE-LMC-T2CEP-010,17.987803,-0.246159,2.147694,-0.075917,-0.748495,-0.020556,0.854472,0.001266,0.001781,0.007313,...,0.015801,0.006009,0.055427,0.005785,0.177524,1.2533,0.4345,0.921205,18.2535,17.7205
OGLE-LMC-T2CEP-018,17.974725,-0.250150,3.214320,-0.079428,1.343289,0.008427,-0.331874,0.001147,0.001634,0.006417,...,0.012026,0.005593,0.044745,0.005599,0.161328,1.2878,0.4035,0.970476,18.2065,17.7025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OGLE-SMC-T2CEP-42,18.225195,0.219891,1.260778,0.052307,0.928054,-0.023112,1.273427,0.001587,0.002264,0.010148,...,0.025495,0.008659,0.094326,0.009103,0.234680,1.2492,0.3928,0.797915,18.7460,18.1190
OGLE-SMC-T2CEP-45,18.125149,-0.217013,2.756931,-0.046731,0.248831,-0.007461,0.762992,0.001079,0.001521,0.007029,...,0.015379,0.004664,0.091066,0.005621,0.414983,1.2533,0.5645,0.773610,18.5065,18.0315
OGLE-SMC-T2CEP-51,18.712840,0.132976,-4.456141,0.033778,2.642192,-0.006404,0.748959,0.001568,0.002231,0.016531,...,0.045855,0.007301,0.145921,0.007265,11.190241,1.4027,0.3801,0.466113,19.2805,18.6705
OGLE-SMC-T2CEP-52,18.153958,-0.196022,-1.381428,-0.032503,-0.441534,0.026221,1.656336,0.001891,0.002731,0.013346,...,0.045178,0.013842,0.172889,0.013537,0.321740,1.4137,0.5152,0.666108,18.7340,18.1170
